Dump: https://colab.research.google.com/drive/19MiTxg6BR-VzhFnpe6yh9V9iq3vf13Nu

Evaluation:
- Perlu scrapping data reviews dari tempat yang belum ada di dataset sebelumnya
Data:

In [ ]:
!pip install tensorflow==2.15.0
!pip install tensorflow-recommenders --no-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
      Successfully uninstalled wrapt-1.16.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstalled keras-3.5.0
  Attempting uninstall: tensorboard
    Found existing installation

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs

import json
from google.colab import drive
from collections import Counter

# Load Dataset

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
places_data = pd.read_csv('/content/drive/MyDrive/voyageur/places_data_merged.csv')
places_data.sample(5)

,Place_Id,Place_Name,Description,Category,City,Price,Rating,Link,Address
1218,1217.0,Bebek Bengil,Bebek Bengil Ubud,Kuliner,Bali,165000.0,43.0,https://www.google.com/maps/place/Bebek+Bengil...,"Jl. Hanoman, Ubud, Kecamatan Ubud, Kabupaten G..."
783,782.0,Pusat Jajanan Malam Raffles Kelapa Gading,Raffles Kelapa Gading - Street Food,Kuliner,Jakarta,0.0,46.0,https://www.google.com/maps/place/Pusat+Jajana...,"RWF7+94X, RT.1/RW.1, Pegangsaan Dua, Kelapa Ga..."
1377,1376.0,Pantai Mertasari,Mertasari Beach,Pantai,Bali,25000.0,44.0,https://www.google.com/maps/place/Pantai+Merta...,"Mertasari Beach, Bali, Indonesia"
18,19.0,Kidzania,KidZania adalah sebuah waralaba swasta Meksiko...,Taman Hiburan,Jakarta,185000.0,46.0,NaN,NaN
230,230.0,Stone Garden Citatah,"Stone Garden, adalah sebutan nama untuk hampa...",Taman Hiburan,Bandung,30000.0,44.0,NaN,NaN


In [ ]:
ratings_data = pd.read_csv('/content/drive/MyDrive/voyageur/ratings.csv')
ratings_data.sample(3)

,Unnamed: 0,User_Id,Place_Id,Place_Ratings,Place_Name,Description,City,Category,Price
7477,7477,225,185,3,Pantai Krakal,Pantai krakal merupakan pantai strategis untuk...,Yogyakarta,Bahari,10000.0
9949,9949,299,409,3,Rumah Batik,Di rumah batik ini terdapat sekitar 2000 lemba...,Surabaya,Budaya,0.0
3825,3825,117,307,4,Perkebunan Teh Malabar,Salah satu perkebunan teh yang cukup terkenal ...,Bandung,Cagar Alam,5000.0


In [ ]:
ratings_data = ratings_data.loc[:, ~ratings_data.columns.str.contains('^Unnamed')]
ratings_data.sample(3)

,User_Id,Place_Id,Place_Ratings,Place_Name,Description,City,Category,Price
9036,272,141,5,Bunker Kaliadem Merapi,Bunker Kaliadem merupakan objek wisata yang te...,Yogyakarta,Cagar Alam,3000.0
1386,43,86,5,Keraton Yogyakarta,Keraton Ngayogyakarta Hadiningrat atau Keraton...,Yogyakarta,Budaya,15000.0
140,5,67,4,Margasatwa Muara Angke,Suaka margasatwa Muara Angke adalah sebuah kaw...,Jakarta,Cagar Alam,25000.0


In [ ]:
def categorize_price(price):
  if 0 <= price < 25:
    return '0-25'
  elif 25 <= price < 50:
    return '25-50'
  elif 50 <= price < 100:
    return '50-100'
  elif 100 <= price < 200:
    return '100-200'
  elif price >= 200:
    return '200+'
  else:
    return 'Unknown'

ratings_data['Price_Category'] = places_data['Price'].apply(categorize_price)

##convert to tensorflow

In [ ]:
places_data['Description'] = places_data['Description'].astype(str)
places_data['Category'] = places_data['Category'].astype(str)
places_data['City'] = places_data['City'].astype(str)

places_data_tf = tf.data.Dataset.from_tensor_slices({
    'Place_Id': tf.cast(places_data['Place_Id'].values, dtype=tf.int64),  # String column for place names
    'Place_Name': tf.convert_to_tensor(places_data['Place_Name'].values, dtype=tf.string),    # String column for links
    'Description': tf.convert_to_tensor(places_data['Description'].values, dtype=tf.string),
    'Category': tf.convert_to_tensor(places_data['Category'].values, dtype=tf.string),  # City as string
    'City': tf.convert_to_tensor(places_data['City'].values, dtype=tf.string),  # Category as string
    'Price': tf.cast(places_data['Price'].values, dtype=tf.float64),  # Fee as float
    'Rating': tf.cast(places_data['Rating'].values, dtype=tf.float64),  # Address as string
})

places_data_tf

<_TensorSliceDataset element_spec={'Place_Id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'Place_Name': TensorSpec(shape=(), dtype=tf.string, name=None), 'Description': TensorSpec(shape=(), dtype=tf.string, name=None), 'Category': TensorSpec(shape=(), dtype=tf.string, name=None), 'City': TensorSpec(shape=(), dtype=tf.string, name=None), 'Price': TensorSpec(shape=(), dtype=tf.float64, name=None), 'Rating': TensorSpec(shape=(), dtype=tf.float64, name=None)}>

In [ ]:
ratings_data['Description'] = ratings_data['Description'].astype(str)
ratings_data['Category'] = ratings_data['Category'].astype(str)
ratings_data['City'] = ratings_data['City'].astype(str)
ratings_data['Price_Category'] = ratings_data['Price_Category'].astype(str)

ratings_data_tf = tf.data.Dataset.from_tensor_slices({
    'User_Id': tf.cast(ratings_data['User_Id'].values, dtype=tf.int64),
    'Place_Id': tf.cast(ratings_data['Place_Id'].values, dtype=tf.int64),  # String column for place names
    'Place_Name': tf.convert_to_tensor(ratings_data['Place_Name'].values, dtype=tf.string),    # String column for links
    'Description': tf.convert_to_tensor(ratings_data['Description'].values, dtype=tf.string),
    'Category': tf.convert_to_tensor(ratings_data['Category'].values, dtype=tf.string),  # City as string
    'City': tf.convert_to_tensor(ratings_data['City'].values, dtype=tf.string),  # Category as string
    'Price': tf.cast(ratings_data['Price'].values, dtype=tf.float64),  # Fee as float
    'Price_Category': tf.convert_to_tensor(ratings_data['Price_Category'].values, dtype=tf.string),  # Category as string
    'Place_Ratings': tf.cast(ratings_data['Place_Ratings'].values, dtype=tf.float64),  # Address as string
})

ratings_data_tf

<_TensorSliceDataset element_spec={'User_Id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'Place_Id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'Place_Name': TensorSpec(shape=(), dtype=tf.string, name=None), 'Description': TensorSpec(shape=(), dtype=tf.string, name=None), 'Category': TensorSpec(shape=(), dtype=tf.string, name=None), 'City': TensorSpec(shape=(), dtype=tf.string, name=None), 'Price': TensorSpec(shape=(), dtype=tf.float64, name=None), 'Price_Category': TensorSpec(shape=(), dtype=tf.string, name=None), 'Place_Ratings': TensorSpec(shape=(), dtype=tf.float64, name=None)}>

In [ ]:
ratings = ratings_data_tf.map(lambda x: {
    "Place_Name": x["Place_Name"],
    'City': x['City'],
    'Category': x['Category'],
    'Price_Category': x['Price_Category']
})

places = places_data_tf.map(lambda x: x["Place_Name"])

In [ ]:
city_unique = np.unique(np.concatenate(list(ratings.batch(64).map(lambda x: x["City"]))))
category_unique = np.unique(np.concatenate(list(ratings.batch(64).map(lambda x: x["Category"]))))
price_unique = np.unique(np.concatenate(list(ratings.batch(64).map(lambda x: x["Price_Category"]))))
place_name_unique = np.unique(np.concatenate(list(places.batch(64))))
print('city_unique:', city_unique, len(city_unique))
print('category_unique:', category_unique, len(category_unique))
print('price_unique:', price_unique, len(price_unique))
print('place_name_unique:', place_name_unique, len(place_name_unique))

city_unique: [b'Bandung' b'Jakarta' b'Semarang' b'Surabaya' b'Yogyakarta' b'nan'] 6
category_unique: [b'Bahari' b'Budaya' b'Cagar Alam' b'Pusat Perbelanjaan' b'Taman Hiburan'
 b'Tempat Ibadah' b'nan'] 7
price_unique: [b'0-25' b'200+' b'Unknown' b'nan'] 4
place_name_unique: [b'(WBF) Wisata Bukit Flora | Agrowisata, Outbound, Edukasi, Anak'
 b'2.28 Klub' b'A Fung' ... b'penida kelingking paramotor'
 b'tanjung kuning' b'tare lagoon'] 1437


# Model

In [ ]:
class UserModel(tf.keras.Model):

  def __init__(self):
    super().__init__()

    self.city_embedding = tf.keras.Sequential([
        tf.keras.layers.StringLookup(
            vocabulary=city_unique, mask_token=None),
        tf.keras.layers.Embedding(len(city_unique) + 1, 32),
    ])

    self.category_embedding = tf.keras.Sequential([
        tf.keras.layers.StringLookup(
            vocabulary=category_unique, mask_token=None),
        tf.keras.layers.Embedding(len(category_unique) + 1, 32),
    ])

    self.price_embedding = tf.keras.Sequential([
        tf.keras.layers.StringLookup(
            vocabulary=price_unique, mask_token=None),
        tf.keras.layers.Embedding(len(price_unique) + 1, 32),
    ])

  def call(self, inputs):
    return tf.concat([
        self.city_embedding(inputs["City"]),
        self.category_embedding(inputs["Category"]),
        self.price_embedding(inputs["Price_Category"]),
    ], axis=1)

In [ ]:
class QueryModel(tf.keras.Model):

  def __init__(self, layer_sizes):
    super().__init__()

    self.embedding_model = UserModel()

    self.dense_layers = tf.keras.Sequential()

    for layer_size in layer_sizes[:-1]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

    for layer_size in layer_sizes[-1:]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size))

  def call(self, inputs):
    feature_embedding = self.embedding_model(inputs)
    return self.dense_layers(feature_embedding)

In [ ]:
class PlaceModel(tf.keras.Model):

  def __init__(self):
    super().__init__()

    max_tokens = 10000

    self.place_embedding = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
          vocabulary=place_name_unique,mask_token=None),
      tf.keras.layers.Embedding(len(place_name_unique) + 1, 32)
    ])

    self.place_vectorizer = tf.keras.layers.TextVectorization(
        max_tokens=max_tokens)

    self.place_text_embedding = tf.keras.Sequential([
      self.place_vectorizer,
      tf.keras.layers.Embedding(max_tokens, 32, mask_zero=True),
      tf.keras.layers.GlobalAveragePooling1D(),
    ])

    self.place_vectorizer.adapt(places)

  def call(self, place):
    return tf.concat([
        self.place_embedding(place),
        self.place_text_embedding(place),
    ], axis=1)

In [ ]:
class CandidateModel(tf.keras.Model):

  def __init__(self, layer_sizes):
    super().__init__()

    self.embedding_model = PlaceModel()

    self.dense_layers = tf.keras.Sequential()

    for layer_size in layer_sizes[:-1]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

    for layer_size in layer_sizes[-1:]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size))

  def call(self, inputs):
    feature_embedding = self.embedding_model(inputs)
    return self.dense_layers(feature_embedding)

In [ ]:
class PlaceDestinationModel(tfrs.models.Model):

  def __init__(self, layer_sizes):
    super().__init__()
    self.query_model = QueryModel(layer_sizes)
    self.candidate_model = CandidateModel(layer_sizes)
    self.task = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=places.batch(128).map(self.candidate_model),
        ),
    )

  def compute_loss(self, features, training=False):

    query_embeddings = self.query_model({
        "City": features["City"],
        "Category": features["Category"],
        "Price_Category": features["Price_Category"],
    })
    place_embeddings = self.candidate_model(features["Place_Name"])

    return self.task(
        query_embeddings, place_embeddings, compute_metrics=not training)

# Training

In [ ]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(len(ratings), seed=42, reshuffle_each_iteration=False)

train = shuffled.take(int(0.8 * len(ratings)))
test = shuffled.skip(int(0.8 * len(ratings))).take(int(0.2 * len(ratings)))

cached_train = train.shuffle(len(ratings)).batch(256)
cached_test = test.batch(512).cache()

In [ ]:
num_epochs = 10

model = PlaceDestinationModel([64])
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

one_layer_history = model.fit(
    cached_train,
    validation_data=cached_test,
    validation_freq=5,
    epochs=num_epochs,
    verbose=0)

accuracy = one_layer_history.history["val_factorized_top_k/top_100_categorical_accuracy"][-1]
print(f"Top-100 accuracy: {accuracy:.2f}.")

Top-100 accuracy: 1.00.


In [ ]:
model.evaluate(cached_test, return_dict=True)

4/4 [==============================] - 1s 188ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0345 - factorized_top_k/top_5_categorical_accuracy: 0.2505 - factorized_top_k/top_10_categorical_accuracy: 0.4475 - factorized_top_k/top_50_categorical_accuracy: 0.9880 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 1646.1698 - regularization_loss: 0.0000e+00 - total_loss: 1646.1698


{'factorized_top_k/top_1_categorical_accuracy': 0.03449999913573265,
 'factorized_top_k/top_5_categorical_accuracy': 0.25049999356269836,
 'factorized_top_k/top_10_categorical_accuracy': 0.44749999046325684,
 'factorized_top_k/top_50_categorical_accuracy': 0.9879999756813049,
 'factorized_top_k/top_100_categorical_accuracy': 1.0,
 'loss': 1531.786376953125,
 'regularization_loss': 0,
 'total_loss': 1531.786376953125}

# Save and Load Model

In [ ]:
model_json = model.to_json()
with open("/content/drive/My Drive/voyageur/model_architecture.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("/content/drive/My Drive/voyageur/model_weights.tf", save_format='tf')

# Itinerary Builder

In [ ]:
index = tfrs.layers.factorized_top_k.BruteForce(model.query_model)

In [ ]:
def get_recommendation_place(index, query_city, query_category, query_price_category, k_num):
  index.index_from_dataset(
  tf.data.Dataset.zip((places.batch(128), places.batch(128).map(model.candidate_model)))
  )
  _, place = index({
      "City": np.array([query_city]),
      "Category": np.array([query_category]),
      "Price_Category": np.array([query_price_category])},
      k=k_num,
  )
  recommended_places = [place.decode('utf-8') for place in place.numpy()[0]]
  return recommended_places

In [ ]:
# Test Function
get_recommendation_place(index, 'Bandung', 'Budaya', '25-50', 10)

['Saung Angklung Mang Udjo',
 'Monumen Bandung Lautan Api',
 'Museum Barli',
 'Museum Konferensi Asia Afrika',
 'Taman Sejarah Bandung',
 'Museum Nike Ardilla',
 'Museum Pendidikan Nasional',
 'Museum Sri Baduga',
 'Museum Gedung Sate',
 'Monumen Perjuangan Rakyat Jawa Barat']

In [ ]:
# Save the model to Google Drive
drive.mount('/content/drive')
model_path = '/content/drive/MyDrive/voyageur/recommendation_model'
tf.saved_model.save(index, model_path)
print(f"Model saved to {model_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Model saved to /content/drive/MyDrive/voyageur/recommendation_model


Attach recommendation with Grup Data Preference

In [ ]:
#disini data user dummy, data user yang nanti dipakai, dikirim dari apk
user_data = pd.DataFrame([
    {
        "user_id": "U001",
        "name": "Ali",
        "preferred_destinations": ["Bali", "Lombok", "Yogyakarta"],
        "preferred_category": ["Pantai", "Budaya", "Kuliner"],
        "budget_range": [1000000, 3000000],
        "available_dates": ["2024-12-01", "2024-12-15"],
    },
    {
        "user_id": "U002",
        "name": "Siti",
        "preferred_destinations": ["Bandung", "Jakarta", "Surabaya"],
        "preferred_category": ["Kuliner", "Edukasi", "Taman"],
        "budget_range": [500000, 1500000],
        "available_dates": ["2024-12-10", "2024-12-20"],
    },
    {
        "user_id": "U003",
        "name": "Budi",
        "preferred_destinations": ["Malang", "Bali", "Yogyakarta"],
        "preferred_category": ["Alam", "Sejarah", "Gunung"],
        "budget_range": [2000000, 4000000],
        "available_dates": ["2024-12-05", "2024-12-25"],
    },
    {
        "user_id": "U004",
        "name": "Nurul",
        "preferred_destinations": ["Lombok", "Medan", "Padang"],
        "preferred_category": ["Pantai", "Kuliner", "Healing"],
        "budget_range": [1500000, 2500000],
        "available_dates": ["2024-12-12", "2024-12-30"],
    },
    {
        "user_id": "U005",
        "name": "Aisyah",
        "preferred_destinations": ["Jakarta", "Bandung", "Semarang"],
        "preferred_category": ["Edukasi", "Taman", "Kuliner"],
        "budget_range": [1000000, 2000000],
        "available_dates": ["2024-12-18", "2024-12-28"],
    },
    {
        "user_id": "U006",
        "name": "Farhan",
        "preferred_destinations": ["Bali", "Batam", "Makassar"],
        "preferred_category": ["Pantai", "Alam", "Healing"],
        "budget_range": [2500000, 5000000],
        "available_dates": ["2024-12-20", "2024-12-30"],
    },
    {
        "user_id": "U007",
        "name": "Dina",
        "preferred_destinations": ["Malang", "Bandung", "Medan"],
        "preferred_category": ["Budaya", "Sejarah", "Kuliner"],
        "budget_range": [1500000, 3000000],
        "available_dates": ["2024-12-01", "2024-12-15"],
    },
    {
        "user_id": "U008",
        "name": "Rezu",
        "preferred_destinations": ["Lombok", "Yogyakarta", "Semarang"],
        "preferred_category": ["Pantai", "Gunung", "Ekstrem"],
        "budget_range": [3000000, 6000000],
        "available_dates": ["2024-12-05", "2024-12-25"],
    },
    {
        "user_id": "U009",
        "name": "Nina",
        "preferred_destinations": ["Bandung", "Jakarta", "Surabaya"],
        "preferred_category": ["Anak", "Taman", "Edukasi"],
        "budget_range": [500000, 2000000],
        "available_dates": ["2024-12-10", "2024-12-20"],
    },
    {
        "user_id": "U010",
        "name": "Raka",
        "preferred_destinations": ["Medan", "Padang", "Makassar"],
        "preferred_category": ["Safari", "Kuliner", "Alam"],
        "budget_range": [2000000, 4000000],
        "available_dates": ["2024-12-15", "2024-12-31"],
    }
])

In [ ]:
group_data = {
    "group_id": "G001",
    "group_name": "Anti Wacana",
    "members": ["U001", "U002", "U003", "U004", "U005", "U006", "U007", "U008", "U009", "U0010"],
    "days_of_trip": 3
}

In [ ]:
def find_common_preferences(user_data):
    # Find the most common destination
    all_destinations = []
    for destinations in user_data['preferred_destinations']:
        all_destinations.extend(destinations)
    common_destination = pd.Series(all_destinations).mode()[0]

    # Find the top 3 most common categories
    all_categories = []
    for categories in user_data['preferred_category']:
        all_categories.extend(categories)
    common_categories = pd.Series(all_categories).value_counts().head(3).index.tolist()

    # Calculate the average budget range
    min_budgets = user_data['budget_range'].apply(lambda x: x[0])
    max_budgets = user_data['budget_range'].apply(lambda x: x[1])
    intermediate_budget = (min_budgets.mean() + max_budgets.mean()) / 2

    # Find most avaiable date
    all_dates = []
    for dates in user_data['available_dates']:
        all_dates.extend(dates)
    date_counts = Counter(all_dates)
    max_count = max(date_counts.values())
    most_common_dates = [date for date, count in date_counts.items() if count == max_count]

    # if no avaiable dates
    if not most_common_dates:
        most_common_dates = ['No available dates']

    if most_common_dates != ['No available dates']:
        sorted_dates = sorted(most_common_dates)
        trip_start_date = sorted_dates[0]
        # Calculate the trip's end date (3 days after the start)
        trip_end_date = pd.to_datetime(trip_start_date) + pd.Timedelta(days=group_data["days_of_trip"] - 1)
        trip_end_date = trip_end_date.strftime('%Y-%m-%d')

    else:
        trip_start_date = "No available dates"
        trip_end_date = "No available dates"

    return common_destination, common_categories, intermediate_budget, most_common_dates, trip_start_date, trip_end_date

In [ ]:
common_destination, common_categories, intermediate_budget, most_common_dates, trip_start_date, trip_end_date = find_common_preferences(user_data)
group_data['most_common_destination'] = common_destination
group_data['most_common_categories'] = common_categories
group_data['average_budget_range'] = intermediate_budget
group_data['most_available_dates'] = most_common_dates
group_data['trip_start_date'] = trip_start_date
group_data['trip_end_date'] = trip_end_date

group_data

{'group_id': 'G001',
 'group_name': 'Anti Wacana',
 'members': ['U001',
  'U002',
  'U003',
  'U004',
  'U005',
  'U006',
  'U007',
  'U008',
  'U009',
  'U0010'],
 'days_of_trip': 3,
 'most_common_destination': 'Bandung',
 'most_common_categories': ['Kuliner', 'Pantai', 'Edukasi'],
 'average_budget_range': 2425000.0,
 'most_available_dates': ['2024-12-15', '2024-12-20'],
 'trip_start_date': '2024-12-15',
 'trip_end_date': '2024-12-17'}

In [ ]:
def destination_budget_threshold(group_data):
  num_destinations = group_data['days_of_trip'] * 4
  #assuming moving cost / destination
  moving_cost = 50000
  threshold = (group_data['average_budget_range'] - (moving_cost * num_destinations)) / num_destinations
  return threshold

In [ ]:
print(group_data['average_budget_range'])
print(destination_budget_threshold(group_data))

2425000.0
152083.33333333334


In [ ]:
def recommend_places_for_group(group_data, index, k_num=20):
    recommendations = []
    budget_threshold = destination_budget_threshold(group_data)
    seen_places = set()  # Keep track of recommended places

    for category in group_data['most_common_categories']:
        # Determine price category based on the average budget
        if budget_threshold < 75000:
            price_category = '0-25'
        elif 75000 <= budget_threshold < 150000:
            price_category = '25-50'
        elif 150000 <= budget_threshold < 300000:
            price_category = '50-100'
        elif 300000 <= budget_threshold < 600000:
            price_category = '100-200'
        else:
            price_category = '200+'

        recommended_places = get_recommendation_place(
            index,
            group_data['most_common_destination'],
            category,
            price_category,
            k_num
        )
        for place in recommended_places:
          if place not in seen_places:
            recommendations.append(place)
            seen_places.add(place) #add to seen place
    return recommendations

In [ ]:
recommended_places = recommend_places_for_group(group_data, index)
recommended_places_df = pd.DataFrame({'Place_Name': recommended_places})
recommended_places_df = pd.merge(recommended_places_df, places_data, on='Place_Name', how='left')
recommended_places_df

,Place_Name,Place_Id,Description,Category,City,Price,Rating,Link,Address
0,Monumen Bandung Lautan Api,257.0,"Monumen Bandung Lautan Api, merupakan monumen ...",Budaya,Bandung,0.0,43.0,NaN,NaN
1,Jalan Braga,212.0,Jalan Braga adalah nama sebuah jalan utama di ...,Budaya,Bandung,0.0,47.0,NaN,NaN
2,Museum Konferensi Asia Afrika,222.0,Museum Konferensi Asia Afrika merupakan salah ...,Budaya,Bandung,0.0,46.0,NaN,NaN
3,Saung Angklung Mang Udjo,325.0,Saung Angklung Udjo (SAU) adalah suatu tempat ...,Budaya,Bandung,75000.0,47.0,NaN,NaN
4,Taman Film,294.0,Taman Film adalah salah satu taman tematik dan...,Budaya,Bandung,0.0,43.0,NaN,NaN
5,Taman Sejarah Bandung,285.0,Taman Sejarah Bandung adalah taman yang terlet...,Budaya,Bandung,0.0,46.0,NaN,NaN
6,Roemah Seni Sarasvati,286.0,Roemah Seni Sarasvati adalah sebuah galeri yan...,Budaya,Bandung,0.0,44.0,NaN,NaN
7,Taman Budaya Jawa Barat,272.0,Taman Budaya Jawa Barat (TBJB) didirikan pada ...,Budaya,Bandung,0.0,43.0,NaN,NaN
8,Kyotoku Floating Market,331.0,"Kyotoku Floating Market Bandung, sebuah wisata...",Budaya,Bandung,175000.0,45.0,NaN,NaN
9,Museum Pendidikan Nasional,290.0,Museum Pendidikan Nasional Universitas Pendidi...,Budaya,Bandung,5000.0,46.0,NaN,NaN


In [ ]:
import random

def build_itinerary(recommended_places, num_itineraries=3, places_per_itinerary=4):
    # Check if there are enough places to generate the itineraries
    if len(recommended_places) < num_itineraries * places_per_itinerary:
        raise ValueError("Not enough recommended places to create itineraries.")

    # Shuffle the list of places to ensure different itineraries each time
    random.shuffle(recommended_places)

    itineraries = []

    # Create the itineraries
    for i in range(num_itineraries):
        # Extract the places for this itinerary
        start_index = i * places_per_itinerary
        end_index = min((i + 1) * places_per_itinerary, len(recommended_places))
        itinerary = recommended_places[start_index:end_index]
        itineraries.append(itinerary)

    return itineraries

In [ ]:
def build_itinerary_choice(places, days_of_trip):
  itinerary_choice_list = []
  for i in range(3):
      itineraries = build_itinerary(places, days_of_trip)
      itinerary_choice_list.append({
          'id': f'CH{i + 1}',  # Add an 'id' to each itinerary choice
          'itinerary': itineraries
      })

  # Convert to a JSON-friendly structure
  itinerary_choices = {
      'itinerary_choices': itinerary_choice_list
  }

  return itinerary_choices

In [ ]:
itinerary_choices = build_itinerary_choice(recommended_places, group_data['days_of_trip'])
itinerary_choices

{'itinerary_choices': [{'id': 'CH1',
   'itinerary': [['Bandros City Tour',
     'Museum Pos Indonesia',
     'Saung Angklung Mang Udjo',
     'Gedung Sate'],
    ['Museum Sri Baduga',
     'Roemah Seni Sarasvati',
     'Taman Film',
     'Kyotoku Floating Market'],
    ['Taman Budaya Jawa Barat',
     'Museum Pendidikan Nasional',
     'Taman Sejarah Bandung',
     'Museum Gedung Sate']]},
  {'id': 'CH2',
   'itinerary': [['Museum Gedung Sate',
     'Roemah Seni Sarasvati',
     'Museum Sri Baduga',
     'Museum Geologi Bandung'],
    ['Museum Barli',
     'Kyotoku Floating Market',
     'Museum Mandala Wangsit Siliwangi',
     'Bandros City Tour'],
    ['Museum Nike Ardilla',
     'Saung Angklung Mang Udjo',
     'Gedung Sate',
     'Jalan Braga']]},
  {'id': 'CH3',
   'itinerary': [['Museum Pendidikan Nasional',
     'Taman Sejarah Bandung',
     'Bandros City Tour',
     'Museum Mandala Wangsit Siliwangi'],
    ['Museum Konferensi Asia Afrika',
     'Taman Budaya Jawa Barat',
     

In [ ]:
group_data['itinerary_choices'] = itinerary_choices
group_data

{'group_id': 'G001',
 'group_name': 'Anti Wacana',
 'members': ['U001',
  'U002',
  'U003',
  'U004',
  'U005',
  'U006',
  'U007',
  'U008',
  'U009',
  'U0010'],
 'days_of_trip': 3,
 'most_common_destination': 'Bandung',
 'most_common_categories': ['Kuliner', 'Pantai', 'Edukasi'],
 'average_budget_range': 2425000.0,
 'most_available_dates': ['2024-12-15', '2024-12-20'],
 'trip_start_date': '2024-12-15',
 'trip_end_date': '2024-12-17',
 'itinerary_choices': {'itinerary_choices': [{'id': 'CH1',
    'itinerary': [['Bandros City Tour',
      'Museum Pos Indonesia',
      'Saung Angklung Mang Udjo',
      'Gedung Sate'],
     ['Museum Sri Baduga',
      'Roemah Seni Sarasvati',
      'Taman Film',
      'Kyotoku Floating Market'],
     ['Taman Budaya Jawa Barat',
      'Museum Pendidikan Nasional',
      'Taman Sejarah Bandung',
      'Museum Gedung Sate']]},
   {'id': 'CH2',
    'itinerary': [['Museum Gedung Sate',
      'Roemah Seni Sarasvati',
      'Museum Sri Baduga',
      'Museum G

In [ ]:
group_data_json = json.dumps(group_data, indent=2)
group_data_json

'{\n  "group_id": "G001",\n  "group_name": "Anti Wacana",\n  "members": [\n    "U001",\n    "U002",\n    "U003",\n    "U004",\n    "U005",\n    "U006",\n    "U007",\n    "U008",\n    "U009",\n    "U0010"\n  ],\n  "days_of_trip": 3,\n  "most_common_destination": "Bandung",\n  "most_common_categories": [\n    "Kuliner",\n    "Pantai",\n    "Edukasi"\n  ],\n  "average_budget_range": 2425000.0,\n  "most_available_dates": [\n    "2024-12-15",\n    "2024-12-20"\n  ],\n  "trip_start_date": "2024-12-15",\n  "trip_end_date": "2024-12-17",\n  "itinerary_choices": {\n    "itinerary_choices": [\n      {\n        "id": "CH1",\n        "itinerary": [\n          [\n            "Bandros City Tour",\n            "Museum Pos Indonesia",\n            "Saung Angklung Mang Udjo",\n            "Gedung Sate"\n          ],\n          [\n            "Museum Sri Baduga",\n            "Roemah Seni Sarasvati",\n            "Taman Film",\n            "Kyotoku Floating Market"\n          ],\n          [\n          